naitive search

In [7]:
class LenghtError(BaseException):
    """исключение для ошибок длины текста или патерна"""
    pass


def get_lenght(string:str)->int:
    """возвращает длину списка"""
    return len(string)

def check_result(result:int)->str:
    """проверяет результат и выводит сообещение"""
    if result == -1:
        print('нету такого в строке')
    else:
        print(f'елемент найден, начало по индексом {result}')

def check_lenght(value:int):
    """проверяет что длина больше 0"""
    return value <= 0

def naive_search(text:str, substring:str)->int:
    """нативный поиск подстроки"""
    substring_lenght = get_lenght(substring)
    text_lenght = get_lenght(text)

    if check_lenght(substring_lenght):
        raise LenghtError('Длина паттерна должна быть больше 0')
    
    if  check_lenght(text_lenght):
        raise LenghtError('Длина текста должна быть больше 0')

    for start_index in range(text_lenght - substring_lenght + 1):
        match_count = 0

        while match_count < substring_lenght:
            if text[start_index + match_count] != substring[match_count]:
                break
            match_count += 1
        
        if match_count == substring_lenght:
            return start_index #возврат индекса начала найденой подстроки
        
    return -1
        

string = 'amiddima'
target = 'dima'

try:
    possition = naive_search(text=string, substring=target)
    check_result(possition)
except LenghtError as e:
    print(f'Ошбика длины: {e}')
    




        



елемент найден, начало по индексом 4


kmp_search

In [15]:
def compute_lps(pattern:str):
    """вычисляет массив LPS (Lingest Prefix Sufix) для алгоритма
    Кнута-Морриса-Пратта
    """
    lenght_pattern = len(pattern)
    lps = [0] * lenght_pattern # массив LPS
    prefix_lenght = 0 #теущая длина свопадающего префикса и суфикса
    current_index = 1 #текущий индекс в строке шаблона

    while current_index < lenght_pattern:
        if pattern[current_index] == pattern[prefix_lenght]:
            prefix_lenght += 1
            lps[current_index] = prefix_lenght
            current_index += 1
        else:
            if prefix_lenght != 0:
                prefix_lenght = lps[prefix_lenght -1]
            else:
                lps[current_index] = 0 
                current_index += 1
    return lps

def kmp_search(text:str, substring):
    """найти первое вхождение в строке"""
    substring_lenght = get_lenght(substring)
    text_lenght = get_lenght(text)

    lps = compute_lps(pattern=substring)

    text_index = 0 #текущий индекс в тексту
    pattern_index = 0 #текущий индекс в шаблоне

    while text_index < text_lenght:
        if substring[pattern_index] == text[text_index]:
            #символы совпадают, переходим к следущему символу
            text_index += 1
            pattern_index += 1
        elif pattern_index != 0:
            # если есть несовпадение, возвращаемся по LPS
            pattern_index = lps[pattern_index-1]
        else:
            # Если несовпадение в начале, переходим к следующему символу текста
            text_index += 1
        
        if pattern_index == substring_lenght:
            return text_index - pattern_index
        
    return -1

def kmp_search_many_ebtries(text:str, substring):
    """найти все вхождения в строке

    Args:
        text (str): текст в котором будет поиск
        substring (str): искомая строка

    Returns:
        :rtype: str
    """
    substring_lenght = get_lenght(substring)
    text_lenght = get_lenght(text)

    lps = compute_lps(pattern=substring)

    text_index = 0 #текущий индекс в тексту
    pattern_index = 0 #текущий индекс в шаблоне
    positions = []

    while text_index < text_lenght:
        if substring[pattern_index] == text[text_index]:
            #символы совпадают, переходим к следущему символу
            text_index += 1
            pattern_index += 1
        elif pattern_index != 0:
            # если есть несовпадение, возвращаемся по LPS
            pattern_index = lps[pattern_index-1]
        else:
            # Если несовпадение в начале, переходим к следующему символу текста
            text_index += 1
        
        if pattern_index == substring_lenght:
            positions.append(text_index - pattern_index)
            pattern_index = lps[pattern_index-1]
    if not positions:
        return -1
    return positions
        
string = 'amid_dima_amid_dima'
target = 'dima'

result = kmp_search_many_ebtries(string, target)
result_ = kmp_search(string, target)
print(result)
print(result_)






[5, 15]
5


boyera myra algo

In [3]:
def build_shift_table(pattern: str):
    """cоздать таблицу сзувов для алгоритма Боэра-Мура"""
    table = {}
    lenght = len(pattern)
    # для каждого символа в подстроке устанавливаем перемещение равное длине подстроки
    for index, char in enumerate(pattern[:-1]):
        table[char] = lenght - index - 1
    # если символа нету в таблице, перемещение будет равно длине подстроки
    table.setdefault(pattern[-1], lenght)
    return table


def boyer_more_search(text, pattern):
    shift_table = build_shift_table(pattern)
    i = len(pattern) - 1
    while i < len(text):
        j = len(pattern) - 1
        # сравнение символов текста и подстроки
        while j >= 0 and text[i] == pattern[j]:
            i -= 1
            j -= 1
        # если  все символы подстроки совпалают, возвращаем индекс в главном текста
        if j < 0:
            return i + 1  # начало подстроки
        # иначи перемещаем индекс у главном тексте на соотв значение из таблицы перемещений
        i += shift_table.get(text[i], len(pattern))
    return -1


def main(boyer_more_search):
    text = "amiddimaamiddima"
    patern = "dima"

    r = boyer_more_search(text, patern)
    return r


print(main(boyer_more_search))

4


robin karp algo

In [4]:
def polynomial_hash(s, base=256, modulus=101):
    """
    Повертає поліноміальний хеш рядка s.
    """
    n = len(s)
    hash_value = 0
    for i, char in enumerate(s):
        power_of_base = pow(base, n - i - 1) % modulus
        hash_value = (hash_value + ord(char) * power_of_base) % modulus
    return hash_value

def rabin_karp_search(main_string, substring):
    # Довжини основного рядка та підрядка пошуку
    substring_length = len(substring)
    main_string_length = len(main_string)

    # Базове число для хешування та модуль
    base = 256
    modulus = 101

    # Хеш-значення для підрядка пошуку та поточного відрізка в основному рядку
    substring_hash = polynomial_hash(substring, base, modulus)
    current_slice_hash = polynomial_hash(main_string[:substring_length], base, modulus)

    # Попереднє значення для перерахунку хешу
    h_multiplier = pow(base, substring_length - 1) % modulus

    # Проходимо крізь основний рядок
    for i in range(main_string_length - substring_length + 1):
        if substring_hash == current_slice_hash:
            if main_string[i:i + substring_length] == substring:
                return i

        if i < main_string_length - substring_length:
            current_slice_hash = (current_slice_hash - ord(main_string[i]) * h_multiplier) % modulus
            current_slice_hash = (current_slice_hash * base + ord(main_string[i + substring_length])) % modulus
            if current_slice_hash < 0:
                current_slice_hash += modulus

    return -1

main_string = "Being a developer is not easy"
substring = "developer"

position = rabin_karp_search(main_string, substring)
if position != -1:
    print(f"Substring found at index {position}")
else:
    print("Substring not found")

Substring found at index 8
